In [4]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize

In [5]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    # Check validity
    if ds['PatientPosition'].value not in ['AP', 'PA']:
        print('Image position ({}) invalid'.format(ds['PatientPosition'].value))
        return None
    elif ds['BodyPartExamined'].value.lower() != 'chest':
        print('Body part ({}) invalid'.format(ds['BodyPartExamined'].value))
        return None
    elif ds['Modality'].value != 'DX':
        print('Image type ({}) invalid'.format(ds['Modality'].value))
    
    return img

    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_mean, img_std, img_size): 
    img = (img - img_mean) / img_std
    
    proc_img = resize(img, img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # Load architecture from JSON
    json_file = open(model_path, 'r')
    model_json = json_file.read()
    json_file.close()
    model = keras.models.model_from_json(model_json)
    
    # Load weights from hdf5
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prediction = (model.predict(img) > thresh)
    
    return prediction 

In [6]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16

'''
I did not use a constant image mean and standard deviation for model training.
I took the mean and std of each image using samplewise scaling with ImageDataGenerator 
(as suggested in Mentor Help), and so to keep inference consistent with training,
here I take the mean and std of each test image separately.
'''
# img_mean = # loads the mean image value they used during training preprocessing
# img_std = # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = 0.5616551041603088

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_mean = img.mean()
    img_std = img.std()
    
    img_proc = preprocess_image(img, img_mean, img_std, IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
[[ True]]
Load file test2.dcm ...
[[ True]]
Load file test3.dcm ...
[[ True]]
Load file test4.dcm ...
Body part (RIBCAGE) invalid
Load file test5.dcm ...
Image type (CT) invalid
[[ True]]
Load file test6.dcm ...
Image position (XX) invalid
